In [ ]:
# lets just determine whether or not we can recreate the best model from the parallel gridsearch

In [1]:
import pickle

class backtest_object:
  def __init__(self,grid_search, X, y, best_estim_preds, final_model):
    self.grid_search = grid_search
    self.X = X
    self.y = y
    self.best_estim_preds = best_estim_preds
    self.final_model = best_gbm

def Acc_Score(y_true,y_pred):
    global ys
    ys.append(y_pred)
    acc = accuracy_score(y_true, y_pred)    
    return acc

# Load the object from the file
with open('backtest_object_10Y_UST_v1_1.pkl', 'rb') as file:
    backtest_object_10Y_UST_v1_1 = pickle.load(file)

In [2]:
X = backtest_object_10Y_UST_v1_1.X
y = backtest_object_10Y_UST_v1_1.y

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer

In [4]:
# Set up the parameter grid for grid search
param_grid = {
    'n_estimators': [10, 20],
    'learning_rate': [0.1],
    'subsample': [0.75],
    'max_depth': [5],
    'min_samples_leaf': [1]
}

# param_grid = {
#     'n_estimators': [50],
#     'learning_rate': [0.3],
#     'subsample': [0.75],
#     'max_depth': [5,15],
#     'min_samples_leaf': [5]
# }

# Initialize the Gradient Boosting Classifier
gbm = GradientBoostingClassifier(random_state=248)


ys = []

def Acc_Score(y_true,y_pred):
    global ys
    ys.append(y_pred)
    acc = accuracy_score(y_true, y_pred)    
    return acc

def scorer():
    return make_scorer(Acc_Score, greater_is_better=True)

# Create the time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=903 - 631, test_size=1,gap=0, max_train_size=None)

# Perform grid search with cross-validation
grid_search = GridSearchCV(gbm, param_grid, cv=tscv, scoring=scorer(), verbose=1, n_jobs=-1)
grid_search.fit(X, y)

# Get the best model from the grid search
best_gbm = grid_search.best_estimator_

cv_results = pd.DataFrame(grid_search.cv_results_)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)
print(f'Accuracy of the Best Hyperparameters: {grid_search.best_score_:.4f}')

Fitting 632 folds for each of 2 candidates, totalling 1264 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 20, 'subsample': 0.75}
Accuracy of the Best Hyperparameters: 0.4778


In [9]:
param_grid2 = {
    'n_estimators': [20],
    'learning_rate': [0.1],
    'subsample': [0.75],
    'max_depth': [5],
    'min_samples_leaf': [1]
}


In [10]:

# Perform grid search with cross-validation
grid_search2 = GridSearchCV(gbm,param_grid2, cv=tscv, scoring=scorer(), verbose=1)#, n_jobs=-1)
grid_search2.fit(X, y)
# Get the best model from the grid search
best_gbm2 = grid_search2.best_estimator_

cv_results2 = pd.DataFrame(grid_search2.cv_results_)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search2.best_params_)
print(f'Accuracy of the Best Hyperparameters: {grid_search2.best_score_:.4f}')

Fitting 632 folds for each of 1 candidates, totalling 632 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 20, 'subsample': 0.75}
Accuracy of the Best Hyperparameters: 0.4778


In [12]:
# compare cv_results
check1 = cv_results.loc[1,'split0_test_score':]
check2 = cv_results2.loc[0,'split0_test_score':]

In [15]:
check1.equals(check2)

True